# Imports

In [ ]:
import numpy as np
import pandas as pd

import matplotlib.pyplot as plt
from matplotlib import cm

from mpl_toolkits.mplot3d import Axes3D

from sklearn.cluster import MeanShift,OPTICS,DBSCAN
from sklearn.cluster import  estimate_bandwidth

from tifffile import tifffile

import pyvista

# Data preparation

## Open the Binary picture

Open and read properties of the file

In [ ]:

# image = tifffile.imread('./Spheroide/Noyau/pred_full_s21_sph2noy1.tif')
image = tifffile.imread('./Spheroide/pred_full_s21_sph1.tif')

# image = tifffile.imread('512.tif')
# type(image) = numpy.ndarray
plt.imshow(image[0])
print(image.shape)
print(type(image))
print(type(image[0]))
print(type(image[0][0]))
print(type(image[0][0][0]))
# print(type(image[0][0][0][0])) if type(of_your_img) != binary

z = image.shape[0]
y = image.shape[1]
x = image.shape[2]

## Read the stack

We create an array of all the pixels white that are supose to represent a cell

In [ ]:
arr = []

for k in range(z):
  print(k,z)
  for j in range(y):
    for i in range(x):
      if(image[k][j][i] >= 128):   #Binarization for 8bits stacks   
        arr.append(np.array([i,j,k]))

arr = np.array(arr)


In [ ]:
print(arr)

## Plot on 3D plan

In [ ]:
data_fig = plt.figure(figsize=(146,146))
ax = data_fig.add_subplot(111, projection ='3d')
ax.scatter(arr[:, 0], arr[:, 1], arr[:, 2], s= 1000, c=arr[:, 2], cmap = plt.get_cmap('hsv'))

plt.show()

# Clustering Technics

## Mean shift clustering

### Train

In [ ]:
# Note that the quantile parameter has to be change manually in order to cluster the data
bandwidth = estimate_bandwidth(arr, quantile=1)
# bandwidth = estimate_bandwidth(arr, quantile=0.015, n_samples=int(len(arr)*0.75))
model = MeanShift(bandwidth=bandwidth, bin_seeding=True)
model.fit(arr)

# Deduce variables from MSC
cluster_centers = model.cluster_centers_
labels = model.labels_
cluster_label = np.unique(labels)
n_clusters = len(cluster_centers)

print("{} cluster found".format(n_clusters))

#Ploting
msc_fig = plt.figure(figsize=(150, 150))
ax = msc_fig.add_subplot(111, projection ='3d')

# plt.ion()

cmap = plt.get_cmap('hsv')
colors = cmap(np.linspace(0, 1.0, n_clusters))

for kind, col in zip(cluster_label, colors):
    my_members = [i for i in range(len(labels)) if labels[i] == kind]
   
    plt.plot(arr[my_members, 0], arr[my_members, 1], arr[my_members, 2],  marker='o', color=col)

plt.show()

## DBSCAN

### Train

In [ ]:
# fig = plt.figure(figsize=(10, 10))
# ax = Axes3D(fig)
# ax.scatter(arr[:,0], arr[:,1], arr[:,2], s=300)
# ax.view_init(azim=200)
# plt.show()

model = DBSCAN(eps=1., min_samples=6, algorithm='ball_tree')
model.fit_predict(arr)
pred = model.fit_predict(arr)

fig = plt.figure(figsize=(10, 10))
ax = Axes3D(fig)
ax.scatter(arr[:,0], arr[:,1], arr[:,2], c=model.labels_, s=3)
ax.view_init(azim=200)
plt.show()

print("number of cluster found: {}".format(len(set(model.labels_))))
print('cluster for each point: ', model.labels_)

In [ ]:

labels = model.labels_
cluster_label = np.unique(labels)
n_clusters = len(cluster_label)
cmap = plt.get_cmap('hsv')
colors = cmap(np.linspace(0, 1.0, n_clusters))

# Make TIFF image of the clustered data

We create a rgb black image

In [ ]:
black_image = np.zeros((z, y, x, 3), dtype=np.uint8)
print(arr.shape)
print(labels.shape)

Append pixels of cells with their color

In [ ]:
for coord,lab in zip(arr,labels):
    if lab != -1 :
        thiscolor = [np.uint8(np.round(255*colors[lab][0])),np.uint8(np.round(255*colors[lab][1])),np.uint8(np.round(255*colors[lab][2]))]
        black_image[coord[2]][coord[1]][coord[0]] = thiscolor

In [ ]:
plt.imshow(black_image[0])

Save the picture fully segmented

In [ ]:
tifffile.imwrite('./DBSCAN/temp.tif' , black_image, photometric='rgb')

In [ ]:
#  convert HSV to RGB for the future Dataframe
# RGB = [np.uint8(np.round(255*colors[0][0])),np.uint8(np.round(255*colors[0][1])),np.uint8(np.round(255*colors[0][2]))] #TODO remove this line if you havemore than one cluster
RGB = [np.uint8(np.round(255*colors[:][0])),np.uint8(np.round(255*colors[:][1])),np.uint8(np.round(255*colors[:][2]))]
print(RGB)

Check if there are similar rounded color (HSV to rouded RGB might make similar colors with a lot of different values)

In [ ]:
for i,elm in enumerate(colors) :
    for i2,elm2 in enumerate(colors) :
        if i!=i2 and elm[0] == elm2[0] and elm[1] == elm2[1] and elm[2] == elm2[2] :
            print(elm , elm2)
        

Making a Dataframe that regroup all the data to facilitate all the manipulations

In [ ]:
df = pd.DataFrame(data=arr, columns=["X","Y","Z"])
df['group'] = labels
df['color'] = [[np.uint8(np.round(255*colors[elm][0])),np.uint8(np.round(255*colors[elm][1])),np.uint8(np.round(255*colors[elm][2]))] for elm in df['group']]
# Convert length from pixels to micrometer
df["Xreal"] = df["X"] * 0.35 
df["Yreal"] = df["Y"] * 0.35 
df["Zreal"] = df["Z"]

df.sample(10)

Isolate real points of a single cell into Numpy arrays for reconstruction

In [ ]:
# Change this variable to print the cell you want to display
number_of_thegroup = 0

mycell = df.loc[df['group'] == number_of_thegroup]
coordonnees= mycell[["Xreal","Yreal","Zreal"]].to_numpy()

number_of_thegroup = 2
mycell2 = df.loc[df['group'] == number_of_thegroup]
coordonnees2= mycell2[["Xreal","Yreal","Zreal"]].to_numpy()

coordonnees.shape


In [ ]:
mycell.head()

Plot the isolated cell

In [ ]:
msc_fig = plt.figure(figsize=(15, 15))
ax = msc_fig.add_subplot(111, projection ='3d')

plt.ion()

cmap = plt.get_cmap('hsv')
ax.scatter(mycell["Xreal"], mycell["Yreal"], mycell["Zreal"],s= 20, c=mycell["Z"], cmap = cmap)
# for ii in range(0,360,3):
#     ax.view_init(elev=10., azim=ii)
#     plt.savefig("./movie/Noyau/movie%d.png" % ((ii+90)%360))


## Reconstruct with Pyvista (Delaunay)

In [ ]:

plot_kwargs = { 'return_viewer': True, 'background': 'white'}
# points is a 3D numpy array (n_points, 3) coordinates of a sphere
cloud = pyvista.PolyData(coordonnees)
cloud2 = pyvista.PolyData(coordonnees2)
# cloud.plot()

# change the alpha parameter based on the distance between two neighboring points 
volume = cloud.delaunay_3d(alpha=1.)
volume2 = cloud2.delaunay_3d(alpha=1.)

shell = volume.extract_geometry()
# shell.plot(plot_kwargs,jupyter_backend='ipygany')
shell2 = volume2.extract_geometry()

merged = shell.merge(shell2) 


merged.plot(plot_kwargs,jupyter_backend='ipygany')
# shell.save('shell.stl') 

In [ ]:
# uncomment for smoothing 
# merged = merged.smooth(n_iter=150)
# merged.plot(plot_kwargs,jupyter_backend='ipygany')

In [ ]:
print(cluster_label)

Make (independantly) and gather all the meshes of each cluster

In [ ]:

for elm in range(n_clusters-1) :
    mycell = df.loc[df['group'] == elm]
    coordonnees= mycell[["Xreal","Yreal","Zreal"]].to_numpy()
    
    # points is a 3D numpy array (n_points, 3) coordinates of a sphere
    cloud = pyvista.PolyData(coordonnees)
    volume = cloud.delaunay_3d(alpha=1.)
    newshell = volume.extract_geometry()
    print(elm, n_clusters)
    if elm == 0 :
        oldshell =newshell
    else :
        oldshell = oldshell.merge(newshell) 



plot_kwargs = { 'return_viewer': True, 'background': 'white'}
oldshell.plot(plot_kwargs,jupyter_backend='ipygany')

In [ ]:
# uncomment for smoothing the whole mesh before saving
# oldshell = oldshell.smooth(n_iter=150)
# oldshell.plot(plot_kwargs,jupyter_backend='ipygany')

In [ ]:
oldshell.save('./DBSCAN/my_mesh.stl') 